# Version Configuration
## Configuration Files
- package dependencies are listed in following files:
    - `requirements_py.txt`: python pkgs
    - `requirements_r.csv`: R pkgs

In [1]:
import glob
import pandas as pd

In [2]:
glob.glob("/home/jovyan/tools/requirements*")

['/home/jovyan/tools/requirements_py.txt',
 '/home/jovyan/tools/requirements_r.csv']

In [3]:
# python pkgs
print("".join(open("/home/jovyan/tools/requirements_py.txt").readlines()[:5]+["..."]))

alabaster==0.7.13
alembic==1.8.1
altair==4.2.0
anyio==3.6.2
argon2-cffi==21.3.0
...


In [4]:
# R pkgs
pd.read_csv("/home/jovyan/tools/requirements_r.csv", index_col=0).head()

,Package,LibPath,Version,Priority,Depends,Imports,LinkingTo,Suggests,Enhances,License,License_is_FOSS,License_restricts_use,OS_type,MD5sum,NeedsCompilation,Built
AnnotationDbi,AnnotationDbi,/opt/conda/lib/R/library,1.60.2,NaN,"R (>= 2.7.0), methods, utils, stats4, BiocGene...","DBI, RSQLite, S4Vectors (>= 0.9.25), stats, KE...",NaN,"hgu95av2.db, GO.db, org.Sc.sgd.db, org.At.tair...",NaN,Artistic-2.0,NaN,NaN,NaN,NaN,no,4.2.2
ape,ape,/opt/conda/lib/R/library,5.7-1,NaN,R (>= 3.2.0),"nlme, lattice, graphics, methods, stats, utils...",Rcpp,"gee, expm, igraph, phangorn",NaN,GPL-2 | GPL-3,NaN,NaN,NaN,NaN,yes,4.2.2
aplot,aplot,/opt/conda/lib/R/library,0.2.2,NaN,R (>= 4.1.0),"ggfun (>= 0.1.3), ggplot2, ggplotify, patchwor...",NaN,ggtree,NaN,Artistic-2.0,NaN,NaN,NaN,NaN,no,4.2.2
askpass,askpass,/opt/conda/lib/R/library,1.1,NaN,NaN,sys (>= 2.1),NaN,testthat,NaN,MIT + file LICENSE,NaN,NaN,NaN,NaN,yes,4.2.1
assertthat,assertthat,/opt/conda/lib/R/library,0.2.1,NaN,NaN,tools,NaN,"testthat, covr",NaN,GPL-3,NaN,NaN,NaN,NaN,no,4.2.1


## Installation & Updating Configuration
- use `lib.sh` to install pkgs according to the configuration files
    - python pkgs are installed via `pip install -r`
    - for R pkgs, following functions are called in `lib.sh`
        - `tools/install_deps.R`: installing pkgs
        - `tools/bioinfo_installer.R`: installing pkgs from `BiocManager`
- use `writelib.sh` to update configuration files
    - python pkgs are listed by `pip list --format=freeze`
    - for R pkgs, `tools/export_deps.R` is called in `writelib.sh`
- `init.sh` will set up everything you need for reproducing virtual env including package installation
### Appendix: Implementation of R scripts
- `tools/install_deps.R`
```R
### install_deps.R ###

deps <- read.csv("./tools/requirements_r.csv", row.names=1)
"%notin%" <- Negate("%in%")
current_pkg <- installed.packages()

for(package in rownames(deps)) {
    if (package %notin% rownames(current_pkg)) {
        remotes::install_version(package, version = deps[package,"Version"], repos="http://cran.ism.ac.jp/")
    } else if (deps[package,"Version"] != current_pkg[package,"Version"]) {
        remotes::install_version(package, version = deps[package,"Version"], repos="http://cran.ism.ac.jp/")
    } else {
        print(sprintf("Requirement is already satisfied: %s (version==%s)", package, deps[package,"Version"]))
    }
}

```
- `tools/bioinfo_installer.R`
```R
### bioinfo_installer.R ###

install.packages("BiocManager", repos="http://cran.ism.ac.jp/")
BiocManager::install("clusterProfiler")
BiocManager::install("AnnotationDbi")
BiocManager::install("org.Hs.eg.db")
BiocManager::install("org.Mm.eg.db")
BiocManager::install("GO.db")
install.packages("optparse", repos="http://cran.ism.ac.jp/")
install.packages("gprofiler2", repos="http://cran.ism.ac.jp/")
```
- `tools/export_deps.R`
```R
### export_deps.R ###

write.csv(installed.packages(), "./tools/requirements_r.csv")
```